In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import random
import os

2025-02-09 08:00:39.427294: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-09 08:00:39.438842: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739106039.452497   52648 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739106039.456501   52648 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-09 08:00:39.470538: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
  for gpu in gpus:
    try:
      print("Enabling GPU: ", gpu)
      tf.config.experimental.set_visible_devices(gpu, 'GPU')
      tf.config.experimental.set_memory_growth(gpu, True)
    except Exception as e:
      print(e)

Enabling GPU:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Enabling GPU:  PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [3]:
val_size = 0.3
rand_seed = random.randint(0, 2**32 - 1)
img_dim = 96
batch_size = 8

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "training"),
  validation_split = val_size,
  subset = "training",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size,
  shuffle = True)

val_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "training"),
  validation_split = val_size,
  subset = "validation",
  seed = rand_seed,
  image_size = (img_dim, img_dim),
  batch_size = batch_size,
  shuffle = True)

Found 1970 files belonging to 2 classes.
Using 1379 files for training.


I0000 00:00:1739106040.824505   52648 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13665 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:02:00.0, compute capability: 8.9


Found 1970 files belonging to 2 classes.
Using 591 files for validation.


In [5]:
image_input = keras.Input(shape = (img_dim, img_dim, 3))
x = keras.layers.Rescaling(1.0 / 255)(image_input)
x = keras.layers.Conv2D(128, kernel_size = (3, 3), activation = 'relu')(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.Conv2D(256, kernel_size = (3, 3), activation = 'relu')(x)
x = keras.layers.MaxPooling2D(pool_size = (2, 2))(x)
x = keras.layers.Dropout(0.01)(x)
feature_output = keras.layers.Flatten()(x)
x = keras.layers.Dense(2048, activation = 'relu')(feature_output)
classification_output = keras.layers.Dense(2)(x)

model = keras.models.Model(image_input, classification_output)

In [6]:
opt = keras.optimizers.Adagrad(learning_rate = 0.01)
model.compile(optimizer = opt,
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 94, 94, 128)    │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 47, 47, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 45, 45, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 123904)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2048)           │   253,757,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         4,098 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 254,060,290 (969.16 MB)

 Trainable params: 254,060,290 (969.16 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
class_names = train_ds.class_names
AUTOTUNE = tf.data.AUTOTUNE
epochs = 16

In [8]:
hist = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs, 
  callbacks = []
)

Epoch 1/16


I0000 00:00:1739106043.639752   52879 service.cc:148] XLA service 0x7eea8c003ca0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739106043.639785   52879 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-02-09 08:00:43.655084: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1739106043.719785   52879 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-02-09 08:00:43.789851: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.5.82. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
2025-02-09 08:00:4

 13/173 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4767 - loss: 0.8813

I0000 00:00:1739106045.638670   52879 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


173/173 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - accuracy: 0.6427 - loss: 0.6307 - val_accuracy: 0.9780 - val_loss: 0.1174
Epoch 2/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9460 - loss: 0.1458 - val_accuracy: 0.9746 - val_loss: 0.0609
Epoch 3/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9444 - loss: 0.1697 - val_accuracy: 0.9932 - val_loss: 0.0347
Epoch 4/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9829 - loss: 0.0546 - val_accuracy: 0.9932 - val_loss: 0.0247
Epoch 5/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9846 - loss: 0.0432 - val_accuracy: 0.9966 - val_loss: 0.0225
Epoch 6/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9904 - loss: 0.0377 - val_accuracy: 0.9966 - val_loss: 0.0204
Epoch 7/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9876 - loss: 0.0308 - val_accuracy: 0.9932 - val_loss: 0.0150
Epoch 8/16
173/173 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9941 - loss: 0.0200 - val_accuracy: 0.996

In [9]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  os.path.join("data", "test"),
  shuffle = False,
  image_size = (img_dim, img_dim)
)

Found 30 files belonging to 2 classes.


In [10]:
model.evaluate(test_ds)

2025-02-09 08:01:31.296199: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_72', 132 bytes spill stores, 132 bytes spill loads

2025-02-09 08:01:31.322223: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_72', 440 bytes spill stores, 356 bytes spill loads



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 883ms/step - accuracy: 1.0000 - loss: 0.0171


[0.017093151807785034, 1.0]